In [ ]:
!pip install -q telebot
!pip install -q pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import numpy as np
import tensorflow as tf

import pydub
import telebot

In [ ]:
mybot = telebot.TeleBot("Enter token here.", parse_mode=None)
labels_friends = ['Abdollah', 'Azra', 'Davood', 'Javad', 'Khadijeh', 'Kiana', 'Mahdi', 'Maryam', 'Matin', 'Melika', 'Mohadeseh', 'Mohammad', 'Mohammad_parvari', 'Mona', 'Nahid', 'Nima', 'Omid', 'Parisa', 'Parsa', 'Sajedeh', 'Shima', 'Tara']
labels_singers = ['Aref', 'Farhad', 'Mohammad Esfahani', 'Mohsen Yeganeh', 'Shadmehr Aghili']
model_friends = tf.keras.models.load_model('/content/drive/MyDrive/Colab_Notebooks/PyLearn7-Assignment59/Audio-Classification/weights/audio_classification.h5')
model_singers = tf.keras.models.load_model('/content/drive/MyDrive/Colab_Notebooks/PyLearn7-Assignment59/Audio-Classification/weights/singers_audio_classification.h5')


In [13]:
@mybot.message_handler(commands=['start'])
def send_welcome(message):
    msg = mybot.send_message(message.chat.id,"Hi "+str(message.chat.first_name) + "\n" +
                             "Welcome to 'pydio classifier bot'" + "\n" +
                             "/start_audio_recognition")


@mybot.message_handler(commands=['start_audio_recognition'])
def help(message):
    os.makedirs("voice", exist_ok=True )
    os.makedirs("input_audio", exist_ok=True)
    os.makedirs("input_audio_chunks", exist_ok=True)

    my_keyboard= telebot.types.ReplyKeyboardMarkup(row_width=0)
    key1 = telebot.types.KeyboardButton("/friends_audio_recognition")
    key2 = telebot.types.KeyboardButton("/singers_audio_recognition")
    my_keyboard.row(key1)
    my_keyboard.row(key2)
    mybot.send_message(message.chat.id ,"Choose one of the keys below:" , reply_markup=my_keyboard)

@mybot.message_handler(commands=['friends_audio_recognition'])
def send_audio(message):
    msg1 = mybot.reply_to(message,"If your name is in the list below, this bot can recognize your sound." + "\n" +
                            "[Abdollah, Azra, Davood, Javad, Khadijeh, Kiana, Mahdi, Maryam,Matin, Melika, Mohadeseh, Mohammad, Mohammad_parvari, Mona, Nahid,Nima, Omid, Parisa, Parsa, Sajedeh, Shima, Tara]" + "\n" +
                            "Please send me a few seconds of your voice.")
    mybot.register_next_step_handler(msg1,recognize_audio_friends)


@mybot.message_handler(commands=['singers_audio_recognition'])
def send_audio(message):
    msg2 = mybot.reply_to(message,"Please send me a few seconds of the voice of one of the singers below." + "\n" +
                          "['Aref', 'Farhad', 'Mohammad Esfahani', 'Mohsen Yeganeh', 'Shadmehr Aghili']")
    mybot.register_next_step_handler(msg2,recognize_audio_singers)


def audio_preprocess(file_path):

    audio = pydub.AudioSegment.from_file(file_path)

    audio = audio.set_sample_width(2)    # convert to 16-bit (2 bytes per sample) to avoid tensorflow error
    audio = audio.set_frame_rate(48000)
    audio = audio.set_channels(1)        # convert stereo audio to mono audio to avoid tensorflow error

    chunks = pydub.silence.split_on_silence(audio, min_silence_len=2000, silence_thresh=-45)
    result = sum(chunks)
    result.export("input_audio/input_audio.wav", format="wav")

    audio = pydub.AudioSegment.from_file("input_audio/input_audio.wav")
    chunks = pydub.utils.make_chunks(audio, 1000)

    for i , chunk in enumerate(chunks):
        if len(chunk) < 1000 :
            continue
        chunk.export(os.path.join("input_audio_chunks", f"voice{i}.wav"), format="wav")

    audio_chunks = []
    for audio_chunk in os.listdir("input_audio_chunks"):
        audio_chunks.append(audio_chunk)

    xs = []
    for audio_chunk in audio_chunks:
        x = tf.io.read_file("input_audio_chunks/"+audio_chunk)
        x, sample_rate = tf.audio.decode_wav(x, desired_channels=1, desired_samples=48000)
        x = tf.squeeze(x, axis=-1)
        x = x[tf.newaxis,...]
        xs.append(x)

    return xs

def recognize_audio_friends(message):

    fileID = message.voice.file_id
    file_info = mybot.get_file(fileID)
    file_path = file_info.file_path
    downloaded_file = mybot.download_file(file_path)

    with open(file_path, 'wb') as new_file:
        new_file.write(downloaded_file)

    xs = audio_preprocess(file_path)

    preds = []
    for x in xs:
        pred = model_friends(x)
        label = np.argmax(pred)
        preds.append(labels_friends[label])
    # print(preds)
    most_common = max(set(preds), key=preds.count)                # Finds the most common element in preds list
    # print(most)
    print("The voice belongs to: ", most_common)

    mybot.reply_to(message, f'The voice belongs to {most_common}')

    shutil.rmtree('voice')                  # clears folder 'voice'
    shutil.rmtree('input_audio')            # clears folder 'input_audio'
    shutil.rmtree('input_audio_chunks')     # clears folder 'input_audio_chunks'

def recognize_audio_singers(message):

    fileID = message.voice.file_id
    file_info = mybot.get_file(fileID)
    file_path = file_info.file_path
    downloaded_file = mybot.download_file(file_path)

    with open(file_path, 'wb') as new_file:
        new_file.write(downloaded_file)

    xs = audio_preprocess(file_path)

    preds = []
    for x in xs:
        pred = model_singers(x)
        label = np.argmax(pred)
        preds.append(labels_singers[label])
    # print(preds)
    most_common = max(set(preds), key=preds.count)                # Finds the most common element in preds list
    # print(most)
    print("The voice belongs to: ", most_common)

    mybot.reply_to(message, f'The voice belongs to {most_common}')

    shutil.rmtree('voice')                  # clears folder 'voice'
    shutil.rmtree('input_audio')            # clears folder 'input_audio'
    shutil.rmtree('input_audio_chunks')     # clears folder 'input_audio_chunks'

mybot.infinity_polling()

The voice belongs to:  Mahdi
The voice belongs to:  Farhad


2025-04-13 10:11:36,242 (__init__.py:1115 MainThread) ERROR - TeleBot: "Infinity polling exception: 'NoneType' object has no attribute 'file_id'"
ERROR:TeleBot:Infinity polling exception: 'NoneType' object has no attribute 'file_id'
2025-04-13 10:11:36,247 (__init__.py:1117 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telebot/__init__.py", line 1110, in infinity_polling
    self.polling(non_stop=True, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/usr/local/lib/python3.11/dist-packages/telebot/__init__.py", line 1198, in polling
    self.__threaded_polling(non_stop=non_stop, interval=interval, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/usr/local/lib/python3.11/dist-packages/telebot/__init__.py", line 1273, in __threaded_polling
    raise e
  File "/usr/local/lib/python3.11/dist-packages/telebot/__init__.py", line 1235, in __threaded_polling
    self.w

The voice belongs to:  Farhad


2025-04-13 10:16:08,669 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2025-04-13 10:16:08,672 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
